1. *Самостоятельно повторить tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
import pandas as pd

Наши новости

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR" -O articles.csv && rm -rf /tmp/cookies.txt 
!wget 'https://drive.google.com/uc?export=download&id=1Q97K9eGrvpbS4ut5CphZa--gJDRqQp2a' -O users_articles.csv

--2022-05-24 16:28:09--  https://docs.google.com/uc?export=download&confirm=t&id=1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR
Resolving docs.google.com (docs.google.com)... 108.177.127.100, 108.177.127.101, 108.177.127.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.127.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sgo6ep7j932ndl5vhq6ngpl7ov9r23p1/1653409650000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download [following]
--2022-05-24 16:28:09--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sgo6ep7j932ndl5vhq6ngpl7ov9r23p1/1653409650000/14904333240138417226/*/1zrdSHN2tq_Hj3YdbwlM3jk87Oct42XpR?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 142.250.145.132, 2a00:1450:4013:c14::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-

In [ ]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [ ]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [ ]:
!pip install razdel pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 36.9 MB/s 


In [ ]:
# предобработка текстов
import re
import numpy as np
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize  # сегментация русскоязычного текста на токены и предложения https://github.com/natasha/razdel
import pymorphy2  # Морфологический анализатор

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

151


In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2022-05-24 16:28:34--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Resolving drive.google.com (drive.google.com)... 108.177.127.113, 108.177.127.138, 108.177.127.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.127.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/td5m5gd1581d40pkvs3nad4kgcrg2hbi/1653409650000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download [following]
--2022-05-24 16:28:34--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/td5m5gd1581d40pkvs3nad4kgcrg2hbi/1653409650000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 142.250.145.132, 2a00:1450:4013:c14::84
Connecting to doc-0s-c0-docs.googleusercontent.com (doc-0s-c0

In [ ]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
    
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [ ]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)
    
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):    
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# Запускаем очистку текста. Будет долго...
news['title'] = news['title'].progress_apply(lambda x: clean_text(x))

  0%|          | 0/27000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Possible nested set at position 39
  
100%|██████████| 27000/27000 [00:50<00:00, 532.14it/s] 


In [ ]:
news['title'].iloc[:10]

0    заместитель председателя правительства рф серг...
1    матч  финала кубка россии по футболу был приос...
2    форвард авангарда томаш заборский прокомментир...
3    главный тренер кубани юрий красножан прокоммен...
4    решением попечительского совета владивостокско...
5    ио главного тренера вячеслав буцаев прокоммент...
6    запорожский металлург дома потерпел разгромное...
7    сборная сша одержала победу над австрией со сч...
8    бывший защитник сборной россии дарюс каспарайт...
9    полузащитник цска зоран тошич после победы над...
Name: title, dtype: object

In [ ]:
# Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 27000/27000 [05:03<00:00, 88.84it/s] 


А теперь в 3 строчки обучим нашу модель

In [ ]:
# сформируем список наших текстов
texts = list(news['title'].values)

# Создадим корпус из списка с текстами
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [ ]:
N_topic = 20  # можно было менять

In [ ]:
from gensim.models import LdaModel

# Обучаем модель на корпусе
lda = LdaModel(common_corpus, num_topics=N_topic, id2word=common_dictionary, passes=2)  # можно было менять

In [ ]:
from gensim.test.utils import datapath

# Сохраняем модель на диск
temp_file = datapath("model.lda")
lda.save(temp_file)

In [ ]:
# Загружаем обученную модель с диска
lda = LdaModel.load(temp_file)

In [ ]:
# Создаем новый корпус документов, которые раньше не видели
other_texts = list(news['title'].iloc[:3])
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(3, 0.11184122),
 (8, 0.043558422),
 (9, 0.52888715),
 (12, 0.19977292),
 (19, 0.0971903)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [ ]:
x = lda.show_topics(num_topics=N_topic, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

# Печатаем только слова
for topic, words in topics_words:
    print(f"topic_{topic}: " + " ".join(words))

topic_0: рынок рубль млн экономика нефть цена рост
topic_1: ребёнок фестиваль мужчина тело мать родитель женщина
topic_2: данные район территория миссия информация обнаружить пострадать
topic_3: всё гражданин научный очень наука россия учёный
topic_4: ракета сша россия космический глава путин мозг
topic_5: украина млрд проект россия развитие российский область
topic_6: турция турецкий иран рейс сигнал астроном предприниматель
topic_7: px японский сократиться операция япония продукция оборудование
topic_8: новый исследование земля технология первый эксперт россия
topic_9: ген ветер век °с команда дождь кость
topic_10: писать медведев всё школа советский газета мозг
topic_11: учёный турист животное планета музей древний москва
topic_12: погибнуть операция житель террорист трагедия запретить пункт
topic_13: газ корабль произойти вода экипаж море борт
topic_14: самолёт военный сша российский россия американский полёт
topic_15: фонд статья университет млн исследование опубликовать дом
topic

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [ ]:
def get_lda_vector(lda, text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]

    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topic):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [ ]:
get_lda_vector(lda, news['title'].iloc[0])

array([0.        , 0.01248514, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05241339, 0.07424345,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.80312651, 0.05115254])

In [ ]:
topic_matrix = pd.DataFrame([get_lda_vector(lda, text) for text in news['title'].values])
topic_matrix.columns = [f'topic_{i}' for i in range(N_topic)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+[f'topic_{i}' for i in range(N_topic)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,6,0.000000,0.012485,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.052405,...,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.803133,0.051153
1,4896,0.264234,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.251197,0.032360
2,4897,0.000000,0.000000,0.0,0.111933,0.0,0.000000,0.0,0.000000,0.043394,...,0.000000,0.000000,0.19947,0.0,0.0,0.000000,0.0,0.0,0.000000,0.097664
3,4898,0.061008,0.094613,0.0,0.492913,0.0,0.000000,0.0,0.125447,0.000000,...,0.000000,0.014929,0.00000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.051124
4,4899,0.000000,0.000000,0.0,0.000000,0.0,0.720059,0.0,0.000000,0.000000,...,0.102428,0.000000,0.00000,0.0,0.0,0.032372,0.0,0.0,0.000000,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [ ]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [ ]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[[f'topic_{i}' for i in range(N_topic)]].values))

In [ ]:
doc_dict[293672]

array([0.        , 0.        , 0.07720033, 0.        , 0.        ,
       0.04964377, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.25767136, 0.        , 0.        , 0.        ,
       0.28358111, 0.        , 0.        , 0.31690347, 0.        ])

In [ ]:
def get_user_embedding(user_articles_list, doc_dict, func):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, axis=0)
    return user_vector

In [ ]:
user_articles_list = users['articles'].iloc[33]

get_user_embedding(user_articles_list, doc_dict, np.mean)

array([0.15978766, 0.00959968, 0.11373204, 0.06072769, 0.15287903,
       0.00605238, 0.00520486, 0.00836241, 0.04839202, 0.        ,
       0.12035329, 0.        , 0.00846514, 0.00363991, 0.13375385,
       0.        , 0.02549465, 0.        , 0.1030633 , 0.03072873])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [ ]:
FUNC = np.mean

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.024179,0.033320,0.069214,0.020167,0.072964,0.101643,0.007651,0.000000,0.002578,...,0.060999,0.127027,0.099911,0.023220,0.000000,0.081992,0.014761,0.021523,0.119118,0.039625
1,u108690,0.056036,0.044634,0.136722,0.106803,0.123699,0.106365,0.001668,0.000000,0.053792,...,0.078475,0.005028,0.058682,0.011920,0.043749,0.008891,0.045348,0.007069,0.070126,0.025202
2,u108339,0.060797,0.023443,0.178919,0.024382,0.051260,0.033499,0.004555,0.013555,0.061261,...,0.022088,0.009914,0.069959,0.077625,0.021047,0.000000,0.175419,0.002055,0.122137,0.029791


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK' -O users_churn.csv

--2022-05-24 16:36:58--  https://drive.google.com/uc?export=download&id=1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK
Resolving drive.google.com (drive.google.com)... 108.177.127.101, 108.177.127.113, 108.177.127.102, ...
Connecting to drive.google.com (drive.google.com)|108.177.127.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m6dnvctarjp98vrcg0kitcipfalu1ig4/1653410175000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download [following]
--2022-05-24 16:36:59--  https://doc-0k-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m6dnvctarjp98vrcg0kitcipfalu1ig4/1653410175000/14904333240138417226/*/1nWKteQgEr9Rl8CwTRY7N2f7igevNH7oK?e=download
Resolving doc-0k-c0-docs.googleusercontent.com (doc-0k-c0-docs.googleusercontent.com)... 142.250.145.132, 2a00:1450:4013:c14::84
Connecting to doc-0k-c0-docs.googleusercontent.com (doc-0k-c0

In [ ]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.024179,0.033320,0.069214,0.020167,0.072964,0.101643,0.007651,0.000000,0.002578,...,0.127027,0.099911,0.023220,0.000000,0.081992,0.014761,0.021523,0.119118,0.039625,0
1,u108690,0.056036,0.044634,0.136722,0.106803,0.123699,0.106365,0.001668,0.000000,0.053792,...,0.005028,0.058682,0.011920,0.043749,0.008891,0.045348,0.007069,0.070126,0.025202,1
2,u108339,0.060797,0.023443,0.178919,0.024382,0.051260,0.033499,0.004555,0.013555,0.061261,...,0.009914,0.069959,0.077625,0.021047,0.000000,0.175419,0.002055,0.122137,0.029791,1


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.15514022, 0.02674447, 0.41232762, 0.15975812, 0.08612848,
       0.02398267, 0.15242723, 0.0514204 , 0.05866819, 0.10385261])

In [ ]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [ ]:
from sklearn.metrics import (f1_score, roc_auc_score, precision_score,
                             classification_report, precision_recall_curve, confusion_matrix)

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.24940536672658006, F-Score=0.714, Precision=0.674, Recall=0.759


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9573044944473516

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.249405,0.714012,0.673913,0.759184,0.957304


2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог)

In [ ]:
FUNC = np.median

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.000000,0.0,0.022645,0.046048,0.015452,0.0,0.000000,0.014383,0.0,...,0.000000,0.050229,0.111912,0.000000,0.000000,0.000000,0.112783,0.074118,0.0,0.000000
1,u108690,0.026530,0.0,0.098608,0.000000,0.079352,0.0,0.014912,0.202948,0.0,...,0.091036,0.011510,0.011357,0.027749,0.006323,0.005463,0.006834,0.037882,0.0,0.018009
2,u108339,0.015161,0.0,0.153543,0.043659,0.040782,0.0,0.007649,0.000000,0.0,...,0.274582,0.023267,0.013008,0.024232,0.025730,0.000000,0.000000,0.056292,0.0,0.043568


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.000000,0.0,0.022645,0.046048,0.015452,0.0,0.000000,0.014383,0.0,...,0.050229,0.111912,0.000000,0.000000,0.000000,0.112783,0.074118,0.0,0.000000,0
1,u108690,0.026530,0.0,0.098608,0.000000,0.079352,0.0,0.014912,0.202948,0.0,...,0.011510,0.011357,0.027749,0.006323,0.005463,0.006834,0.037882,0.0,0.018009,1
2,u108339,0.015161,0.0,0.153543,0.043659,0.040782,0.0,0.007649,0.000000,0.0,...,0.023267,0.013008,0.024232,0.025730,0.000000,0.000000,0.056292,0.0,0.043568,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.05147989, 0.05956307, 0.33620559, 0.55797399, 0.10104689,
       0.04906116, 0.24036655, 0.05715048, 0.07860949, 0.04248218])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.2618990735725252, F-Score=0.727, Precision=0.675, Recall=0.788


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9646979475550903

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.277892,0.726937,0.663300,0.804082,0.962323
1,median,0.261899,0.726930,0.674825,0.787755,0.964698



3. Повторить п.2, но используя уже не медиану, а max


In [ ]:
FUNC = np.max

In [ ]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, doc_dict, FUNC))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.059428,0.033769,0.311545,0.119189,0.066352,0.167494,0.043208,0.241189,0.176661,...,0.476130,0.242260,0.184943,0.130034,0.229331,0.026306,0.461148,0.322678,0.028463,0.131295
1,u108690,0.249385,0.034385,0.181064,0.022222,0.147014,0.036683,0.132362,0.316068,0.088778,...,0.485771,0.187045,0.132405,0.319676,0.118310,0.057443,0.056915,0.072654,0.022296,0.100535
2,u108339,0.114573,0.021893,0.335829,0.128384,0.098983,0.056686,0.043475,0.000000,0.014791,...,0.532330,0.145295,0.044027,0.120085,0.331589,0.012908,0.014923,0.124789,0.232184,0.237563


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.059428,0.033769,0.311545,0.119189,0.066352,0.167494,0.043208,0.241189,0.176661,...,0.242260,0.184943,0.130034,0.229331,0.026306,0.461148,0.322678,0.028463,0.131295,0
1,u108690,0.249385,0.034385,0.181064,0.022222,0.147014,0.036683,0.132362,0.316068,0.088778,...,0.187045,0.132405,0.319676,0.118310,0.057443,0.056915,0.072654,0.022296,0.100535,1
2,u108339,0.114573,0.021893,0.335829,0.128384,0.098983,0.056686,0.043475,0.000000,0.014791,...,0.145295,0.044027,0.120085,0.331589,0.012908,0.014923,0.124789,0.232184,0.237563,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([0.00863778, 0.02597381, 0.89057964, 0.69861275, 0.09309694,
       0.02162438, 0.02595133, 0.11566657, 0.03321705, 0.0316266 ])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.38016440636075044, F-Score=0.717, Precision=0.715, Recall=0.718


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9544973544973545

In [ ]:
metrics_df = metrics_df.append({
    'model': FUNC.__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.277892,0.726937,0.663300,0.804082,0.962323
1,median,0.261899,0.726930,0.674825,0.787755,0.964698
2,amax,0.380164,0.716904,0.715447,0.718367,0.954497


4. *Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (взяв список новостей пользователя)
	- подсказка 1: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал
	- подсказка 2: нужен именно idf, как вес.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
users['articles_str'] = users['articles'].apply(lambda x: x.replace('[','').replace(']', '').replace(',', ''))

users['articles_str'].iloc[0]

'293672 293328 293001 293622 293126 1852'

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(users['articles_str'])

TfidfVectorizer()

In [ ]:
idf = pd.DataFrame({'article_id': tfidf.get_feature_names_out(),
                    'idf': tfidf.idf_})

idf

,article_id,idf
0,10,8.888710
1,100,7.907880
2,1000,8.041412
3,1001,8.888710
4,1002,8.888710
...,...,...
14776,995,8.377884
14777,996,8.195562
14778,997,8.601027
14779,998,9.294175


In [ ]:
def get_user_embedding_idf(user_articles_list, doc_dict):
    user_articles_list = eval(user_articles_list)
    
    user_vector = np.zeros((len(user_articles_list), N_topic))
    for i, doc_id in enumerate(user_articles_list):
        try:
            weight = idf[idf['article_id'] == str(doc_id)]['idf'].values[0]
        except Exception as e:
            weight = 0
        user_vector[i] = doc_dict[doc_id] * weight

    user_vector = np.median(user_vector, axis=0)
    return user_vector

In [ ]:
from tqdm import tqdm
tqdm.pandas()

user_embeddings = pd.DataFrame([i for i in users['articles'].progress_apply(lambda x: get_user_embedding_idf(x, doc_dict))])
user_embeddings.columns = [f'topic_{i}' for i in range(N_topic)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(N_topic)]]
user_embeddings.head(3)

100%|██████████| 8000/8000 [01:14<00:00, 107.63it/s]


,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,u105138,0.00000,0.170712,0.000000,1.210370,0.000000,0.655122,0.271639,0.314213,0.000000,...,0.781570,0.0,0.000000,0.00000,0.000000,0.000000,0.431723,0.075428,0.000000,0.0
1,u108690,0.13054,0.742963,0.315954,1.274439,0.839233,0.416907,0.246556,0.000000,0.058172,...,0.000000,0.0,0.233407,0.00000,0.066105,0.000000,0.477224,0.000000,0.000000,0.0
2,u108339,0.05678,0.087128,0.250914,1.753166,0.744982,0.319510,0.695659,0.000000,0.050291,...,0.463058,0.0,0.048645,0.24566,0.000000,0.042763,0.488616,0.044294,0.088801,0.0


In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,churn
0,u105138,0.00000,0.170712,0.000000,1.210370,0.000000,0.655122,0.271639,0.314213,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.431723,0.075428,0.000000,0.0,0
1,u108690,0.13054,0.742963,0.315954,1.274439,0.839233,0.416907,0.246556,0.000000,0.058172,...,0.0,0.233407,0.00000,0.066105,0.000000,0.477224,0.000000,0.000000,0.0,1
2,u108339,0.05678,0.087128,0.250914,1.753166,0.744982,0.319510,0.695659,0.000000,0.050291,...,0.0,0.048645,0.24566,0.000000,0.042763,0.488616,0.044294,0.088801,0.0,1


In [ ]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(N_topic)]], 
                                                    X['churn'], random_state=0)

In [ ]:
model = LogisticRegression()
# обучим 
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# наши прогнозы для тестовой выборки
preds = model.predict_proba(X_test)[:, 1]
preds[:10]

array([1.06964663e-01, 1.72833030e-04, 9.97245969e-01, 2.05977195e-01,
       9.34439502e-05, 4.20087782e-04, 4.79674536e-03, 3.25228636e-03,
       2.54923459e-02, 1.20870550e-02])

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.2938956533053592, F-Score=0.840, Precision=0.786, Recall=0.902


In [ ]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.9861596604453747

In [ ]:
metrics_df = metrics_df.append({
    'model': 'idf_median',
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)


5. Сформировать на выходе единую таблицу, сравнивающую качество 2/3 разных метода получения эмбедингов пользователей: median, max, idf_mean по метрикам roc_auc, precision, recall, f_score


In [ ]:
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,mean,0.268411,0.679317,0.634752,0.730612,0.939869
1,median,0.263708,0.715328,0.646865,0.800000,0.959812
2,amax,0.379813,0.724211,0.747826,0.702041,0.953579
3,idf_median,0.293896,0.840304,0.786477,0.902041,0.986160


6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных